# Observer Pattern

The observer pattern is a software design pattern in which an object, called the subject, maintains a list of its dependents, called observers, and notifies them of state changes, typically by calling one of their methods. It is mainly used to implement distributed event handling systems.

In [2]:
use std::sync::{Arc, Weak};

pub trait Observer {
    type Subject;

    fn update(&self, subject: &Self::Subject);
}

pub trait Observable {
    type ObserverPtr;

    fn add_observer(&mut self, observer: Self::ObserverPtr);
    fn remove_observer(&mut self, observer: Self::ObserverPtr);
    fn notify(&self);
}

In [3]:
pub struct Subject {
    observers: Vec<Weak<dyn Observer<Subject = Self>>>, // Using Weak - Subject does not participate in ownership
    state: String,
}

impl Observable for Subject {
    type ObserverPtr = Arc<dyn Observer<Subject = Self>>;

    fn add_observer(&mut self, observer: Self::ObserverPtr) {   
        self.observers.push(Arc::downgrade(&observer));
    }

    fn remove_observer(&mut self, observer: Self::ObserverPtr) {
        self.observers
            .retain(|o| !o.ptr_eq(&Arc::downgrade(&observer)));
    }

    fn notify(&self) {
        self.observers
            .iter()
            .flat_map(|o| o.upgrade())
            .for_each(|o| o.update(&self));
    }
}

impl Subject {
    fn new(state: &str) -> Self {
        Self {
            observers: Vec::new(),
            state: state.into(),
        }
    }

    fn set_state(&mut self, state: &str) {
        if self.state == state {
            return;
        }

        self.state = state.into();
        self.notify();
    }

    fn state(&self) -> &str {
        &self.state
    }
}

In [4]:
struct ConcreteObserver {
    name: String,
}

impl ConcreteObserver {
    fn new(name: &str) -> Arc<Self> {
        Arc::new(Self { name: name.into() })
    }
}

impl Observer for ConcreteObserver {
    type Subject = Subject;

    fn update(&self, subject: &Self::Subject) {
        println!("Observing subject with state={:?} in {}", subject.state(), self.name);
    }
}

In [5]:
fn main() {
    let mut subject = Subject::new("initial state");

    let observer1 = ConcreteObserver::new("observer1");
    let observer2 = ConcreteObserver::new("observer2");

    subject.add_observer(observer1.clone());
    subject.add_observer(observer2.clone());

    subject.set_state("new state - A");
    subject.set_state("new state - B");

    subject.remove_observer(observer1);

    subject.set_state("new state - C");
    subject.set_state("new state - D");
}

main();

Observing subject with state="new state - A" in observer1
Observing subject with state="new state - A" in observer2
Observing subject with state="new state - B" in observer1
Observing subject with state="new state - B" in observer2
Observing subject with state="new state - C" in observer2
Observing subject with state="new state - D" in observer2


# Callbacks

In [4]:
type Callback<TResult = ()> = Box<dyn Fn() -> TResult>;
type CallbackWithArg<T, TResult = ()> = Box<dyn Fn(T) -> TResult>;

In [5]:
struct Printer {
    id: String,
    on_start: Option<Callback>,
    on_print: Option<CallbackWithArg<String>>,
    on_stop: Option<Callback>,
}

impl Printer {
    fn new(id: &str) -> Self {
        Self {
            id: id.into(),
            on_start: None,
            on_print: None,
            on_stop: None,
        }
    }

    fn on_start(&mut self, callback: Callback) {
        self.on_start = Some(callback);
    }

    fn on_print(&mut self, callback: CallbackWithArg<String>) {
        self.on_print = Some(callback);
    }

    fn on_stop(&mut self, callback: Callback) {
        self.on_stop = Some(callback);
    }

    fn start(&self) {
        if let Some(ref on_start) = self.on_start {
            on_start();
        }

        println!("Printer {} started", self.id);
    }

    fn print(&self, text: &str) {
        self.start();

        if let Some(ref on_print) = self.on_print {
            on_print(text.into());
        }

        println!("Printer {} printed: {}", self.id, text);

        self.stop();
    }

    fn stop(&self) {
        if let Some(ref on_stop) = self.on_stop {
            on_stop();
        }

        println!("Printer {} finished", self.id);
    }
}

In [6]:
let mut printer = Printer::new("PRN_1");

printer.on_start(Box::new(|| {
    println!("Log: Printer started");
}));

printer.on_print(Box::new(|text| {
    println!("Log: Printing - {}", text);
}));

printer.on_stop(Box::new(|| {
    println!("Log: Printer stopped");
}));

printer.print("Hello, World!");

Log: Printer started
Printer PRN_1 started
Log: Printing - Hello, World!
Printer PRN_1 printed: Hello, World!
Log: Printer stopped
Printer PRN_1 finished


In [7]:
struct Signal<Arg> {
    slots: Vec<CallbackWithArg<Arg>>,
}

impl<Arg: Copy + Clone> Signal<Arg> {
    fn new() -> Self {
        Self { slots: Vec::new() }
    }

    fn connect(&mut self, slot: CallbackWithArg<Arg>) {
        self.slots.push(slot);
    }

    fn emit(&self, arg: Arg) {
        self.slots.iter().for_each(|slot| slot(arg));
    }
}

In [8]:
let mut signal = Signal::<f64>::new();

signal.connect(Box::new(|arg| {
    println!("Signal emitted with arg: {}", arg);
}));

signal.connect(Box::new(|arg| {
    println!("Signal called with arg: {}", arg);
}));

signal.emit(3.14);

Signal emitted with arg: 3.14
Signal called with arg: 3.14


In [23]:
use std::rc::Rc;
use std::cell::RefCell;

// A temperature sensor that notifies observers when the temperature changes.
pub struct TemperatureSensor {
    temperature: f64,
    pub on_temperature_changed: Signal<f64>,
}

impl TemperatureSensor {
    pub fn new() -> Self {
        Self {
            temperature: 0.0,
            on_temperature_changed: Signal::new(),
        }
    }

    pub fn set_temperature(&mut self, temperature: f64) {
        if self.temperature == temperature {
            return;
        }

        self.temperature = temperature;
        self.on_temperature_changed.emit(temperature);
    }
}

// A simple logger that stores log messages.
pub struct Logger {
    logs: Vec<String>
}

impl Logger {
    pub fn log(&mut self, msg: String) {
        self.logs.push(msg);
    }
}

// A display that shows temperature changes.
pub struct Display;

impl Display {
    pub fn update_display(&self, temperature: f64) {
        println!("Updating display: Current temperature is {:.2}°C", temperature);
    }
}

///////////////////////////////////////////////////////////////////////
let mut temp_sensor = TemperatureSensor::new();
let mut logger = Rc::new(RefCell::new(Logger { logs: vec![] }));
let mut display = Rc::new(RefCell::new(Display));

let rc_display = Rc::clone(&display);
temp_sensor.on_temperature_changed.connect(Box::new(move |temp| {
    display.borrow().update_display(temp);
}));

let rc_logger = Rc::clone(&logger);
temp_sensor.on_temperature_changed.connect(Box::new(move |temp| { 
    rc_logger.borrow_mut().log(format!("Temperature changed: {:.2}°C", temp))
}));

temp_sensor.set_temperature(25.0);
temp_sensor.set_temperature(30.0);
temp_sensor.set_temperature(25.0);
temp_sensor.set_temperature(24.0);

for log in &logger.borrow().logs {
    println!("Log entry: {}", log);
}

Updating display: Current temperature is 25.00°C
Updating display: Current temperature is 30.00°C
Updating display: Current temperature is 25.00°C
Updating display: Current temperature is 24.00°C
Log entry: Temperature changed: 25.00°C
Log entry: Temperature changed: 30.00°C
Log entry: Temperature changed: 25.00°C
Log entry: Temperature changed: 24.00°C


()